In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
from tensorflow.keras import layers
from tqdm import tqdm
import matplotlib.pyplot as plt
import pathlib
import os

AUTOTUNE = tf.data.experimental.AUTOTUNE
tf.keras.backend.clear_session()  # For easy reset of notebook state.

tf.__version__
tf.executing_eagerly()


True

In [2]:

ds = tfds.load(name="imagenette", with_info=True, split=["train","validation"])
ds_train=ds[0][0]
ds_test=ds[0][1]
assert isinstance(ds_train, tf.data.Dataset)

def normalize(x):
    y = {'image': tf.image.convert_image_dtype(x['image'], tf.float32), 'label': x['label']}
    y = (tf.image.resize(y['image'], (224,224)), y['label'])
    return y
    
ds_train = ds_train.map(lambda x: normalize(x))
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(100)
ds_train = ds_train.batch(8)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = ds_test.map(
    normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(8)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [35]:
#conv2D with bias and relu activation

class CustomConvLayer(layers.Layer):

    def __init__(self, weights, mask, biases, stride=1, padding='SAME'):
        
        super(CustomConvLayer, self).__init__()
        self.w = weights
        self.m = mask
        self.b = biases
        self.s = stride
        self.p = padding

        
    def call(self, inputs):
        x = tf.nn.conv2d(inputs, tf.multiply(self.w, self.m), strides=[1, self.s, self.s, 1], padding=self.p)
        x = tf.nn.bias_add(x, self.b)
        return tf.nn.relu(x)

    
class ResNetBlock(tf.keras.layers.Layer):
    def __init__(self, input_channels=3 ,output_channels = 64, stride=1):
        super(ResNetBlock, self).__init__()
        self.stride = stride
        #self.conv1 = layers.Conv2D(filters, 3, strides=(self.stride, self.stride), activation='relu', padding='same')
        self.conv1 = CustomConvLayer(tf.Variable(tf.keras.initializers.glorot_uniform()([3, 3, input_channels, output_channels])),
                                     tf.Variable(tf.ones([3, 3, input_channels, output_channels]), trainable=False),
                                     tf.Variable(tf.zeros([output_channels]))
                                    )
        self.bn1 = layers.BatchNormalization()
        #self.conv2 = layers.Conv2D(filters, 3, padding='same')
        self.conv2 = CustomConvLayer(tf.Variable(tf.keras.initializers.glorot_uniform()([3, 3, output_channels, output_channels])),
                                     tf.Variable(tf.ones([3, 3, output_channels, output_channels]), trainable=False),
                                     tf.Variable(tf.zeros([output_channels]))
                                    )
        self.bn2 = layers.BatchNormalization()
        if stride == 2:
            self.conv3 = CustomConvLayer(tf.Variable(tf.keras.initializers.glorot_uniform()([1, 1, output_channels, output_channels])),
                                         tf.Variable(tf.ones([1, 1, output_channels, output_channels]), trainable=False),
                                         tf.Variable(tf.zeros([output_channels])),
                                         stride=self.stride)
            self.bn3 = layers.BatchNormalization()
        self.add1 = layers.Add()
    
    def call(self, inputs, training=False):
        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = self.conv2(x)
        x = self.bn2(x, training=training)
        if self.stride == 2:
            inputs = self.conv3(x)
            inputs = self.bn3(x)
        return tf.nn.relu(self.add1([x, inputs]))
    

class CustomResNetModel(tf.keras.Model):
    def __init__(self):
        super(CustomResNetModel, self).__init__()
        self.conv1 = layers.Conv2D(64, 7, strides=(2, 2), padding='same')
        self.pool1 = layers.MaxPool2D(pool_size=(3,3), strides=(2,2), padding='same')
        self.res_block1 = ResNetBlock(64, 64)
        self.res_block2 = ResNetBlock(64, 64)
        self.res_block3 = ResNetBlock(64, 64)
        self.res_block4 = ResNetBlock(64, 128, 2)
        self.res_block5 = ResNetBlock(128, 128)
        self.res_block6 = ResNetBlock(128, 128)
        self.res_block7 = ResNetBlock(128, 128)
        self.res_block8 = ResNetBlock(128, 256, 2)
        self.res_block9 = ResNetBlock(256, 256)
        self.res_block10 = ResNetBlock(256, 256)
        self.res_block11 = ResNetBlock(256, 256)
        self.res_block12 = ResNetBlock(256, 256)
        self.res_block13 = ResNetBlock(256, 256)
        self.res_block14 = ResNetBlock(256 ,512, 2)
        self.res_block15 = ResNetBlock(512, 512)
        self.res_block16 = ResNetBlock(512, 512)
        self.pool2 = layers.GlobalAveragePooling2D()
        self.dense1 = layers.Dense(1000, activation='relu')
        self.dense2 = layers.Dense(10, activation='softmax')

    def call(self,inputs):
        x = self.conv1(inputs)
        x = self.pool1(x)
        x = self.res_block1(x)
        x = self.res_block2(x)
        x = self.res_block3(x)
        x = self.res_block4(x)
        x = self.res_block5(x)
        x = self.res_block6(x)
        x = self.res_block7(x)
        x = self.res_block8(x)
        x = self.res_block9(x)
        x = self.res_block10(x)
        x = self.res_block11(x)
        x = self.res_block12(x)
        x = self.res_block13(x)
        x = self.res_block14(x)
        x = self.res_block15(x)
        x = self.res_block16(x)
        x = self.pool2(x)
        x = self.dense1(x)
        
        return self.dense2(x)

In [36]:
model = CustomResNetModel()

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)


In [ ]:
model.fit(
    x=ds_train,
    #steps_per_epoch=1,
    epochs=3,
    validation_data=ds_test,
)
model.run_eagerly = False

Epoch 1/3


    126/Unknown - 4413s 35s/step - loss: 2.7042 - accuracy: 0.1528

In [20]:
glorot_uniform_initializer = tf.keras.initializers.glorot_uniform()
x = glorot_uniform_initializer((3, 3, 64, 64))

In [21]:
tf.Variable(x)

<tf.Variable 'Variable:0' shape=(3, 3, 64, 64) dtype=float32, numpy=
array([[[[ 2.02629566e-02,  1.92947835e-02,  7.02714175e-02, ...,
          -3.31248082e-02,  3.07162926e-02, -3.53846252e-02],
         [ 5.78053445e-02, -5.52489609e-03,  1.43863410e-02, ...,
          -5.95670193e-02,  1.13598257e-03,  7.13178813e-02],
         [ 6.40948117e-03,  2.04993039e-03,  4.79052067e-02, ...,
           5.50265312e-02, -3.59847844e-02,  4.27871495e-02],
         ...,
         [ 3.46322060e-02, -5.16249835e-02,  4.76861000e-03, ...,
           2.12328136e-03,  2.21860930e-02,  4.81134057e-02],
         [ 2.24393383e-02, -5.06244153e-02,  7.13824332e-02, ...,
           3.89303938e-02, -3.92184630e-02,  7.07585514e-02],
         [ 2.33934969e-02, -6.55349642e-02, -5.38643450e-02, ...,
           6.50467575e-02,  4.06134725e-02,  4.72745970e-02]],

        [[-6.14162236e-02,  6.56358749e-02,  2.12041289e-02, ...,
           5.50430566e-02, -2.01923065e-02,  6.58020377e-04],
         [-5.447566